In [12]:
import json
from datetime import datetime, date
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import os

In [83]:
def get_date(date_str):
    return datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%S.%fZ").date()

def convert_time_to_seconds(time_str):
    time = datetime.strptime(time_str, "%Y-%m-%dT%H:%M:%S.%fZ").time()
    return time.hour * 3600 + time.minute * 60 + time.second

In [84]:
def get_game_info(game,tournament,region,stage):
    game_info = game[0]
    game_end = game[-1]
    
    blue = pd.DataFrame({"side":["blue"]})
    red = pd.DataFrame({"side":["red"]})
    
    start_time = convert_time_to_seconds(game_info["eventTime"])
    end_time = convert_time_to_seconds(game_end["eventTime"])
    date = get_date(game_info["eventTime"])

    blue["date"] = date
    blue["start_time"] = start_time
    red["date"] = date
    red["start_time"] = start_time

    blue["end_time"] = end_time
    red["end_time"] = end_time

    blue["duration"] = blue["end_time"] - blue["start_time"]
    red["duration"] = blue["duration"]
    
    patch = game_info["gameVersion"]
    blue["patch"] =patch
    red["patch"] =patch
    
    participants =game_info["participants"]
    roles = ["top","jng","mid","adc","sup","top","jng","mid","adc","sup"]
    
    for player,role in zip(participants,roles):
        summonerName =player["summonerName"].split(" ")
        team_tag = summonerName.pop(0)
        player_name = " ".join(summonerName)
        if player["teamID"]==100:
            side = blue
        else:
            side = red
        side["team"]= team_tag
        side[role]= player_name
        if not team_tag in regions or regions[team_tag]==None:
            regions[team_tag]=region
    blue["opponent"]= red["team"]
    red["opponent"]= blue["team"]
    for role in ["top","jng","mid","adc","sup"]:
        blue[f"enemy_{role}"]=red[role]
        red[f"enemy_{role}"]=blue[role]
    blue_tag = blue["team"][0]
    red_tag = red["team"][0]
    blue["region"]=regions.get(blue_tag)
    red["region"]=regions.get(red_tag)
    blue["opponent_region"]=regions.get(red_tag)
    red["opponent_region"]=regions.get(blue_tag)
    blue["stage"]=stage
    red["stage"]=stage
    

    if game_end["winningTeam"]== 100:
        blue["result"]=1
        red["result"]=0
    else:
        blue["result"]=0
        red["result"]=1
    return(blue,red)

In [85]:
def get_status_updates(game,blue,red):
    start_time = blue["start_time"]
    roles = ["top","jng","mid","adc","sup","top","jng","mid","adc","sup"]
    for event in game:
        event_time = int(convert_time_to_seconds(event["eventTime"]) - start_time)
        for time in ["300"]: #can add more times, 300 sec for 5 mins game time
            if (event["eventType"] == "stats_update") and (event_time == int(time)):
                print("True", event_time)
                participants = event["participants"]
                variables =["totalGold","level", "shutdownValue"]
                stats= ["MINIONS_KILLED","NEUTRAL_MINIONS_KILLED","NEUTRAL_MINIONS_KILLED_YOUR_JUNGLE","NEUTRAL_MINIONS_KILLED_ENEMY_JUNGLE","CHAMPIONS_KILLED","NUM_DEATHS", "ASSISTS", "VISION_SCORE", "WARD_KILLED", "WARD_PLACED", "TOTAL_DAMAGE_DEALT", "TOTAL_DAMAGE_DEALT_TO_CHAMPIONS", "TOTAL_DAMAGE_TAKEN", "BASIC_PINGS", "COMMAND_PINGS","DANGER_PINGS","GET_BACK_PINGS","RETREAT_PINGS", "ON_MY_WAY_PINGS", "ASSIST_ME_PINGS", "ENEMY_MISSING_PINGS", "PUSH_PINGS", "ALL_IN_PINGS", "HOLD_PINGS", "BAIT_PINGS", "VISION_CLEARED_PINGS", "ENEMY_VISION_PINGS", "NEED_VISION_PINGS"]
                for player,role in zip(participants,roles):
                    if player["teamID"]==100:
                        side = blue
                    else:
                        side = red
                    for var in variables:
                        side[f"{role}_{var}_{time}"] = player[var]
            
                    stat_list = player["stats"]
                    for stat in stat_list:
                        if stat["name"] in stats:
                            name = stat["name"]
                            side[f"{role}_{name}_{time}"] = stat["value"]
    return(blue,red)

In [81]:
#dictionary for team regions
regions = {}
df = pd.DataFrame()
directory = "C:/GLOBAL POWER RANKINGS/games/"
for folder in os.listdir(directory):
    tournament = folder.split("_")[0]
    if tournament != "msi" or tournament !="worlds" or tournament !="midseason":
        #setting the region for teams who play in the tournaments (excluding international events)
        region = tournament
    else:
        region = None
    for stage in os.listdir(f"{directory}{folder}"):
        print(f" processing {directory}{folder}/{stage}")
        for games in os.listdir(f"{directory}{folder}/{stage}"):
            with open(f"{directory}{folder}/{stage}/{games}", "r") as json_file:
                game = json.load(json_file)
                blue,red = get_game_info(game,tournament,region,stage)
                blue,red = get_status_updates(game,blue,red)
                df =pd.concat([df,blue,red])
                
df.head()

 processing C:/GLOBAL POWER RANKINGS/games/lec_spring_2023/Groups
True 300
 processing C:/GLOBAL POWER RANKINGS/games/lec_spring_2023/Playoffs
True 300
 processing C:/GLOBAL POWER RANKINGS/games/lec_spring_2023/Regular Season
True 300


,side,date,start_time,end_time,duration,patch,team,top,jng,mid,...,sup_ON_MY_WAY_PINGS_300,sup_ASSIST_ME_PINGS_300,sup_ENEMY_MISSING_PINGS_300,sup_PUSH_PINGS_300,sup_ALL_IN_PINGS_300,sup_HOLD_PINGS_300,sup_BAIT_PINGS_300,sup_VISION_CLEARED_PINGS_300,sup_ENEMY_VISION_PINGS_300,sup_NEED_VISION_PINGS_300
0,blue,2023-04-08 17:11:21.362,61881,63897,2016,13.6.500.291,MAD,Chasy,Elyoya,Nisqy,...,2,0,1,0,0,0,0,0,0,0
0,red,2023-04-08 17:11:21.362,61881,63897,2016,13.6.500.291,VIT,Photon,Bo,Perkz,...,0,0,0,0,0,0,0,0,0,0
0,blue,2023-04-17 17:10:15.279,61815,63823,2008,13.6.500.291,VIT,Photon,Bo,Perkz,...,1,0,3,0,0,0,0,0,2,0
0,red,2023-04-17 17:10:15.279,61815,63823,2008,13.6.500.291,BDS,Adam,Sheo,nuc,...,1,0,2,0,0,0,0,0,1,0
0,blue,2023-03-20 19:15:02.328,69302,71130,1828,13.4.497.3450,AST,Finn,113,LIDER,...,6,2,1,0,0,0,0,0,3,0


In [39]:
pd.pivot_table(df, index = 'result', columns = 'team', values = 'region' ,aggfunc ='count')

team,AST,BDS,FNC,G2,KOI,MAD,SK,TH,VIT,XL
result,,,,,,,,,,
0,8,5,9,9,10,14,9,7,9,7
1,8,16,6,13,7,17,4,3,11,2
